<a href="https://colab.research.google.com/github/ayush5/-CS69002_9A_18CS60R26/blob/master/MLP_Tutorial_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Review Sentiment Analysis

## Import Header files

In [0]:
import torch
import pandas as pd
import numpy as np

In [3]:
print('Welcome to Computing Lab 2. The current PyTorch version is ', torch.__version__)

Welcome to Computing Lab 2. The current PyTorch version is  1.0.1.post2


## Load the dataset and visualize


There are 3 ways to do this:

(a)  From Github (Files < 25MB)

The easiest way to upload a CSV file is from your GitHub repository. Click on the dataset in your repository, then click on View Raw. Copy the link to the raw dataset and store it as a string variable called url in Colab as shown below (a cleaner method but it’s not necessary). The last step is to load the url into Pandas read_csv to get the dataframe.


```
url = 'copied_raw_GH_link'
df1 = pd.read_csv(url)
# Dataset is now stored in a Pandas Dataframe

```

---


(b) From a local drive

To upload from your local drive, start with the following code:

```
from google.colab import files
uploaded = files.upload()
```

It will prompt you to select a file. Click on “Choose Files” then select and upload the file. Wait for the file to be 100% uploaded. You should see the name of the file once Colab has uploaded it.

Finally, type in the following code to import it into a dataframe (make sure the filename matches the name of the uploaded file).
```
import io
df2 = pd.read_csv(io.BytesIO(uploaded['Filename.csv']))
# Dataset is now stored in a Pandas Dataframe
```

---


(c) From Google drive

This the most complicated of the three methods. Follow this [link](https://medium.freecodecamp.org/how-to-transfer-large-files-to-google-colab-and-remote-jupyter-notebooks-26ca252892fa) to learn more.

In [0]:
url = 'https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Train_20K.csv'
df_train = pd.read_csv(url, sep='\t')
url="https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Test_5K.csv"
df_test= pd.read_csv(url, sep="\t")

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
type(uploaded), uploaded.keys(), type(uploaded['check.csv'])

In [0]:
import io
df = pd.read_csv(io.StringIO(uploaded['check.csv'].decode('utf-8')), sep='\t')
df.head()

## Pandas

[Pandas](https://pandas.pydata.org/) is a high-performance, easy-to-use data structure and data analytics tool for Python.

Tutorials:

*   [Link 1](https://pandas.pydata.org/pandas-docs/stable/getting_started/tutorials.html)
*   [Link 2](https://www.tutorialspoint.com/python_pandas)
*   [Link 3](https://www.dataquest.io/blog/pandas-python-tutorial/)



In [5]:
print('Number of Negative movie reviews', len(df[df['label']==0]))
print('Number of Positive movie reviews', len(df[df['label']==1]))

NameError: ignored

## Data pre-processing
The first step when building a neural network model is getting your data into the proper form to feed into the network. We'll need to encode each word with an integer. We'll also want to clean it up a bit.

You can use any type of pre-processing that you see fit, based on the task. For the tutorial, i will show just one pre-processing step: **lowercasing**.

In [6]:
text_reviews = df_train['text'].astype(str).tolist()
text_labels = df_train['label'].astype(int)

text_reviews = [x.lower() for x in text_reviews]
text_reviews[0], text_labels[0]

('john waters has given us a genuinely enjoyable film. this certainly isn\'t without its shocking waters-esque moments, but it is tamer than his older culty stuff, such as "pink flamingoes". "pecker" harkens back to john\'s early mainstream stage in that it reminds the viewer of the same kind of humor that was evident in "polyester". overall, a really fun comedy with some great moments!',
 1)

In [7]:
len(text_reviews)

17999

In [8]:
text_reviews = df_test['text'].astype(str).tolist()
text_labels = df_test['label'].astype(int)

text_reviews = [x.lower() for x in text_reviews]
text_reviews[0], text_labels[0]

("as always this is an inaccurate picture of the homeless. tv told a lot of lies about panhandlers in the early 1990s and made everyone look bad, and claimed we all made over $100 a day when $20-40 a day was much closer to reality. when someone drove by where i held up a sign offering to work, and offered me work, i actually went and took the work if i was physically able.and if i would been offered the $100,000 id damned sure invested in in apt prepaid for at least 2 years, and kept most in the bank and still left myself $10-20000 for nl $1-2 and $2-5 cash games at the casinos. i usually always win and could win decent if i just had a bankroll. instead i win about $1000 a month is all playing in always minimum buying in due to not wanting to risk losing it all. i was only homeless cause i didn't wanna risk spending all my money and going broke, sometimes i had over $1000-2000 in my sock while i slept outside. anyone wanting to talk contact sevencard2003 on yahoo messenger.i admit i wa

In [9]:
len(text_reviews[0])

1554

## Creating Bag Of Word (BOW) representation of sentences.

In [0]:
def generate_word_ids(dataset):
  word_to_ix = {}
  for sent in dataset:
      for word in sent.split():
          if word not in word_to_ix:
              word_to_ix[word] = len(word_to_ix)
  return word_to_ix

In [11]:
generate_word_ids(['welcome to computing lab 2'])

{'2': 4, 'computing': 2, 'lab': 3, 'to': 1, 'welcome': 0}

In [12]:
train_text= df_train['text'].astype(str).tolist()
train_label=df_train['label'].astype(int)
test_text=df_test['text'].astype(str).tolist()
test_label=df_test['label'].astype(int)
print (train_label[0])

1


In [0]:
#data = [("me gusta comer en la cafeteria", "SPANISH"),
        ("Give it to me", "ENGLISH"),
        ("No creo que sea una buena idea", "SPANISH"),
        ("No it is not a good idea to get lost at sea", "ENGLISH")]

#test_data = [("Yo creo que si", "SPANISH"),
             ("it is lost on me", "ENGLISH")]

In [13]:
word_to_ix = generate_word_ids(train_text+test_text)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2
print(VOCAB_SIZE)

266290


## Intro to PyTorch

In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

lin = nn.Linear(5,3)
data1 = Variable(torch.randn(10,5)) # create a 10 * 5 matrix of random vars

In [15]:
data1, data1.size()

(tensor([[-0.4974,  0.4396, -0.7581,  1.0783,  0.8008],
         [ 1.6806,  0.3559, -0.6866,  0.6105,  1.3347],
         [-0.2316,  0.0418, -0.2516,  0.8599, -0.3097],
         [-0.3957, -0.2234,  1.7174,  0.3189, -0.4245],
         [ 0.3057, -0.7746,  0.0349,  0.3211, -0.8798],
         [-0.6011, -1.2742,  2.1228, -1.2347, -0.4879],
         [-1.4181,  0.8963,  0.0780,  0.5258,  0.3466],
         [-0.1973, -1.0546,  1.2780,  0.1453,  0.2311],
         [ 0.0087, -0.1423,  0.5750, -0.6417, -2.2064],
         [-0.7508,  2.8140,  0.3598, -0.0898,  0.4584]]), torch.Size([10, 5]))

In [16]:
lin(data1), lin(data1).size()

(tensor([[ 0.3739,  0.1674, -0.1029],
         [ 0.8355,  0.0414,  0.8471],
         [ 0.2831,  0.6893, -0.1183],
         [-0.2887,  1.0737,  0.3888],
         [-0.0317,  0.9654, -0.0386],
         [-1.4234,  0.7988,  0.2472],
         [-0.0414,  0.1357, -0.2777],
         [-0.6189,  0.8737,  0.3820],
         [-0.2207,  0.9984, -0.2671],
         [ 0.6052, -0.4270,  0.2221]], grad_fn=<AddmmBackward>),
 torch.Size([10, 3]))

In [17]:
lin.weight.size()

torch.Size([3, 5])

**BE VERY CAREFUL WITH THE DIMENSIONS**

In [18]:
F.relu(lin(data1))  # Activation Function

tensor([[0.3739, 0.1674, 0.0000],
        [0.8355, 0.0414, 0.8471],
        [0.2831, 0.6893, 0.0000],
        [0.0000, 1.0737, 0.3888],
        [0.0000, 0.9654, 0.0000],
        [0.0000, 0.7988, 0.2472],
        [0.0000, 0.1357, 0.0000],
        [0.0000, 0.8737, 0.3820],
        [0.0000, 0.9984, 0.0000],
        [0.6052, 0.0000, 0.2221]], grad_fn=<ReluBackward0>)

## Model Definition for the BOWClassifier

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 50)
    self.lin2 = nn.Linear(50, num_labels)
    
    
  def forward(self, x):
    z=F.softmax(self.lin(x))
    return F.softmax(self.lin2(z))

##Generate the BOW Vectors

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence.split():
        if word not in word_to_ix:
            raise ValueError('Word',word,' not present in the dictionary. Sorry!')
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])


In [0]:
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE)

In [22]:
for param in bow.parameters():
    print(param,param.size())

Parameter containing:
tensor([[ 1.1761e-03, -4.5986e-04,  1.1085e-03,  ...,  1.8404e-04,
         -1.8414e-03,  1.8761e-03],
        [-1.6271e-03, -9.7494e-04, -1.3696e-03,  ...,  5.1725e-04,
          7.4541e-04,  2.9641e-05]], requires_grad=True) torch.Size([2, 266290])
Parameter containing:
tensor([-0.0003, -0.0005], requires_grad=True) torch.Size([2])


### Let's check with a sample case how our model is working

In [23]:
sample_data=test_text[0]
sample_label = test_label[0]
bow_vector = torch.autograd.Variable(make_bow_vector(sample_data, word_to_ix))
logprobs = bow(bow_vector)
print()
print(logprobs)


tensor([[0.5053, 0.4947]], grad_fn=<SoftmaxBackward>)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


In [24]:
label_to_ix = {0: 0, 1: 1}
ix_to_label = {v: k for k, v in label_to_ix.items()}

label_to_ix, ix_to_label

({0: 0, 1: 1}, {0: 0, 1: 1})

In [25]:

for y,instance in enumerate(test_text):
    label=test_label[y]
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = bow(bow_vec)
    print(logprobs)
    pred = np.argmax(logprobs.data.numpy())
    print('prediction: {}'.format(ix_to_label[pred]))
    print('actual: {}'.format(label))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


tensor([[0.5053, 0.4947]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[0.5035, 0.4965]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[0.4973, 0.5027]], grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 0
tensor([[0.5020, 0.4980]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[0.5011, 0.4989]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[0.5050, 0.4950]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 1
tensor([[0.4822, 0.5178]], grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1
tensor([[0.5125, 0.4875]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 1
tensor([[0.5011, 0.4989]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[0.4994, 0.5006]], grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1
tensor([[0.5024, 0.4976]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 1
tensor([[0.4860, 0.5140]], grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1
tensor([[0.4966, 0.5034]], grad_fn=<SoftmaxBackward>)
prediction

In [0]:
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [38]:
# the training loop
for epoch in range(10):
    print(epoch)
    for y,instance in enumerate(train_text):
        # get the training data
        label=train_label[y]
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix))
        label = Variable(make_target(label, label_to_ix))
        probs = bow(bow_vec) # forward pass
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()
        


0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


1
2
3
4
5
6
7
8
9


In [41]:
total_count=0
correct_count=0
print('--- AFTER TRAINING ---')
for y,instance in enumerate(test_text):
    label=test_label[y]
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = bow(bow_vec)
    print(logprobs)
    pred = np.argmax(logprobs.data.numpy())
    print('prediction: {}'.format(ix_to_label[pred]))
    print('actual: {}'.format(label))
    if label==ix_to_label[pred]:
      correct_count+=1
    total_count+=1

--- AFTER TRAINING ---
tensor([[0.7723, 0.2277]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[1.0000e+00, 5.4353e-23]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[1.0000e+00, 1.0805e-19]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[0.0018, 0.9982]], grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 0
tensor([[1.0000e+00, 4.3224e-23]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[7.7165e-21, 1.0000e+00]], grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1
tensor([[1.4094e-16, 1.0000e+00]], grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1
tensor([[4.2940e-29, 1.0000e+00]], grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1
tensor([[1.0000e+00, 5.3039e-07]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[6.8034e-16, 1.0000e+00]], grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1
tensor([[1.0000e+00, 8.9978e-13]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 1
tensor([[6.0419e-17, 1.0000e+00]], grad_fn=<Sof

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


tensor([[9.9997e-01, 3.0026e-05]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[0.9703, 0.0297]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[1.0000e+00, 3.4867e-10]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[6.1688e-10, 1.0000e+00]], grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 0
tensor([[1.0000e+00, 2.2000e-21]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[2.3499e-05, 9.9998e-01]], grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 0
tensor([[1.2215e-33, 1.0000e+00]], grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1
tensor([[3.6799e-11, 1.0000e+00]], grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1
tensor([[1.0000e+00, 5.3274e-27]], grad_fn=<SoftmaxBackward>)
prediction: 0
actual: 0
tensor([[0.3331, 0.6669]], grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 0
tensor([[2.1814e-15, 1.0000e+00]], grad_fn=<SoftmaxBackward>)
prediction: 1
actual: 1
tensor([[1.1228e-19, 1.0000e+00]], grad_fn=<SoftmaxBackward>)
predicti

In [42]:
print (correct_count/total_count)

0.8066


In [0]:
torch.save(bow,'model.json')

from google.colab import files
files.download("model.json")

In [0]:
from google.colab import files
temp_test = files.upload()


In [0]:
torch.load(io.BytesIO(temp_test['model.json']))